# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

OperationalError: FATAL:  database "sparkifydb" does not exist


In [5]:
conn.set_session(autocommit=True)

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [8]:
for root, dirs, files in os.walk('data/song_data'):
    files = glob.glob(os.path.join(root,'*.json'))
    #for f in files :
        #all_files.append(os.path.abspath(f))
    print(files)

[]
[]
[]
['data/song_data/A/A/A/TRAAAAW128F429D538.json', 'data/song_data/A/A/A/TRAAAVO128F93133D4.json', 'data/song_data/A/A/A/TRAAADZ128F9348C2E.json', 'data/song_data/A/A/A/TRAAAMO128F1481E7F.json', 'data/song_data/A/A/A/TRAAAVG12903CFA543.json', 'data/song_data/A/A/A/TRAAARJ128F9320760.json', 'data/song_data/A/A/A/TRAAAMQ128F1460CD3.json', 'data/song_data/A/A/A/TRAAAPK128E0786D96.json', 'data/song_data/A/A/A/TRAAAFD128F92F423A.json', 'data/song_data/A/A/A/TRAAAEF128F4273421.json', 'data/song_data/A/A/A/TRAAABD128F429CF47.json']
[]
[]
['data/song_data/A/A/B/TRAABCL128F4286650.json', 'data/song_data/A/A/B/TRAABJV128F1460C49.json', 'data/song_data/A/A/B/TRAABJL12903CDCF1A.json', 'data/song_data/A/A/B/TRAABVM128F92CA9DC.json', 'data/song_data/A/A/B/TRAABYN12903CFD305.json', 'data/song_data/A/A/B/TRAABYW128F4244559.json', 'data/song_data/A/A/B/TRAABXG128F9318EBD.json', 'data/song_data/A/A/B/TRAABDL12903CAABBA.json', 'data/song_data/A/A/B/TRAABLR128F423B7E3.json', 'data/song_data/A/A/B/T

In [6]:
song_file_str = ""

for i in range(0, len(song_files)):
    if i == 0:
        with open(song_files[i]) as song_content:
            for line in song_content.readlines():
                song_file_str = song_file_str + line
    else:
        with open(song_files[i]) as song_content:
            for line in song_content.readlines():
                song_file_str = song_file_str + ',' + line

song_file_df = pd.read_json(song_file_str, lines=True)        
        


In [7]:
#clean the song_file_df by replaceing np.nan and '' with 'None' 
song_file_df = song_file_df.fillna(value='None')
song_file_df = song_file_df.replace('', 'None')

In [9]:
#drop duplicated rows
song_file_df = song_file_df.drop_duplicates(keep='first')


In [10]:
song_file_df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.8262,"Morris Plains, NJ",-74.48,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
1,ARB29H41187B98F0EF,41.8841,Chicago,-87.6324,Terry Callier,342.56934,1,SOGNCJP12A58A80271,Do You Finally Need A Friend,1972
2,ARGIWFO1187B9B55B7,None,None,None,Five Bolt Main,225.09669,1,SOPSWQW12A6D4F8781,Made Like This (Live),0
3,ARH4Z031187B9A71F2,40.732,"Newark, NJ",-74.1742,Faye Adams,156.39465,1,SOVYKGO12AB0187199,Crazy Mixed Up World,1961
4,AR5KOSW1187FB35FF4,49.8039,Dubai UAE,15.4749,Elena,269.58322,1,SOZCTXZ12AB0182364,Setanta matins,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = song_file_df.loc[:,['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data_list = song_data.values.tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
for row in song_data_list:
    cur.execute(song_table_insert, row)
    conn.commit()

In [13]:
cur.execute("select * from song")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('SOBBUGU12A8C13E95D', 'Setting Fire to Sleeping Giants', 'ARMAC4T1187FB3FA4C', 2004, 207.77751)
('SOGNCJP12A58A80271', 'Do You Finally Need A Friend', 'ARB29H41187B98F0EF', 1972, 342.56934)
('SOPSWQW12A6D4F8781', 'Made Like This (Live)', 'ARGIWFO1187B9B55B7', 0, 225.09669)
('SOVYKGO12AB0187199', 'Crazy Mixed Up World', 'ARH4Z031187B9A71F2', 1961, 156.39465)
('SOZCTXZ12AB0182364', 'Setanta matins', 'AR5KOSW1187FB35FF4', 0, 269.58322)
('SOGXHEG12AB018653E', 'It Makes No Difference Now', 'AR0RCMP1187FB3F427', 1992, 133.32853)
('SOYTPEP12AB0180E7B', 'Twist and Shout', 'ARAJPHH1187FB5566A', 1964, 164.80608)
('SOBLGCN12AB0183212', 'James (Hold The Ladder Steady)', 'AR47JEX1187B995D81', 1985, 124.86485)
('SOBZBAZ12A6D4F8742', 'Spanish Grease', 'AROUOZZ1187B9ABE51', 1997, 168.25424)
('SOOLYAZ12A6701F4A6', 'Laws Patrolling (Album Version)', 'AREBBGV1187FB523D2', 0, 173.66159)
('SOZQDIU12A58A7BCF6', 'Superconfidential', 'ARNTLGG11E2835DDB9', 0, 338.31138)
('SONYPOM12A8C13B2D7', 'I Think My Wife

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
artist_data = song_file_df.loc[:,['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude"']]
artist_data_list = artist_data.replace(np.nan, 'None').values.tolist()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
for row in artist_data_list:
    cur.execute(artist_table_insert, row)
    conn.commit()

In [16]:
cur.execute("select * from artist")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('ARMAC4T1187FB3FA4C', 'The Dillinger Escape Plan', 'Morris Plains, NJ', '40.82624', 'None')
('ARB29H41187B98F0EF', 'Terry Callier', 'Chicago', '41.88415', 'None')
('ARGIWFO1187B9B55B7', 'Five Bolt Main', 'None', 'None', 'None')
('ARH4Z031187B9A71F2', 'Faye Adams', 'Newark, NJ', '40.73197', 'None')
('AR5KOSW1187FB35FF4', 'Elena', 'Dubai UAE', '49.80388', 'None')
('AR0RCMP1187FB3F427', 'Billie Jo Spears', 'Beaumont, TX', '30.08615', 'None')
('ARAJPHH1187FB5566A', 'The Shangri-Las', 'Queens, NY', '40.7038', 'None')
('AR47JEX1187B995D81', 'SUE THOMPSON', 'Nevada, MO', '37.83721', 'None')
('AROUOZZ1187B9ABE51', 'Willie Bobo', 'New York, NY [Spanish Harlem]', '40.79195', 'None')
('AREBBGV1187FB523D2', "Mike Jones (Featuring CJ_ Mello & Lil' Bran)", 'Houston, TX', 'None', 'None')
('ARNTLGG11E2835DDB9', 'Clp', 'None', 'None', 'None')
('ARDNS031187B9924F0', 'Tim Wilson', 'Georgia', '32.67828', 'None')
('AR1Y2PT1187FB5B9CE', 'John Wesley', 'Brandon', '27.94017', 'None')
('ARGUVEV1187B98BA17', '

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files = get_files('data/log_data')

In [18]:
log_file_str = ""

for i in range(0, len(log_files)):
    if i == 0:
        with open(log_files[i]) as log_content:
            for line in log_content.readlines():
                log_file_str = log_file_str + line
    else:
        with open(log_files[i]) as log_content:
            for line in log_content.readlines():
                log_file_str = log_file_str + '\n' + line

log_file_str_replaced = log_file_str.replace('}\n{', '},{')
log_file_df = pd.read_json(log_file_str_replaced, lines=True)        
        


In [19]:
# clean song_file_df by replacing np.nan and '' with 'None'
log_file_df = log_file_df.fillna(value='None')
log_file_df = log_file_df.replace('', 'None')

In [20]:
#drop duplicated rows
log_file_df = log_file_df.drop_duplicates(keep='first')

In [23]:
log_file_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,None,free,"Klamath Falls, OR",GET,Home,1.54108e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.1604,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.54027e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.159,free,"Klamath Falls, OR",PUT,NextSong,1.54108e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.226,free,"Klamath Falls, OR",PUT,NextSong,1.54108e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,None,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.54022e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [24]:
filtered_log_file_df = log_file_df[log_file_df['page'] == 'NextSong']
filtered_log_file_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.1604,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.54027e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.159,free,"Klamath Falls, OR",PUT,NextSong,1.54108e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.226,free,"Klamath Falls, OR",PUT,NextSong,1.54108e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.175,free,"Klamath Falls, OR",PUT,NextSong,1.54108e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.539,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.54022e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


In [25]:
exact_time = pd.to_datetime(filtered_log_file_df['ts'], unit='ms')
exact_time.head()

1   2018-11-12 02:37:38.796
2   2018-11-12 02:37:44.796
3   2018-11-12 02:42:21.796
5   2018-11-12 02:45:52.796
6   2018-11-12 02:47:22.796
Name: ts, dtype: datetime64[ns]

In [26]:
#Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data
timestamp = filtered_log_file_df['ts']
hour = exact_time.dt.hour
day = exact_time.dt.day
weekofYear = exact_time.dt.weekofyear
month = exact_time.dt.month
year = exact_time.dt.year
weekday = exact_time.dt.weekday
time_data_df = pd.concat([timestamp, hour, day, weekofYear, month, year, weekday], axis=1)

In [27]:
col_labels = ['timestamp', 'hour', 'day', 'weekofYear', 'month', 'year', 'weekday']
time_data_df.columns = col_labels
time_data_df = time_data_df.drop_duplicates(keep='first')
time_data_df.head()

,timestamp,hour,day,weekofYear,month,year,weekday
1,1541990258796,2,12,46,11,2018,0
2,1541990264796,2,12,46,11,2018,0
3,1541990541796,2,12,46,11,2018,0
5,1541990752796,2,12,46,11,2018,0
6,1541990842796,2,12,46,11,2018,0


In [28]:
time_data_list = time_data_df.values.tolist()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [29]:
for row in time_data_list:
    cur.execute(time_table_insert, row)
    conn.commit()

In [ ]:
try: 
    cur.execute("SELECT * FROM time;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
user_df = log_file_df.loc[:,['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates(keep='first')

In [32]:
user_list = user_df.values.tolist()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [33]:
for row in user_list:
    cur.execute(user_table_insert, row)
    conn.commit()

In [ ]:
try: 
    cur.execute("""SELECT * FROM "user" """)
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [44]:
#read two tables and get the data needed (song and artist) from postgres data
#song_select = 'select song.song_id, song.artist_id, artist.name, song.title, song.duration from song join artist on song.artist_id = artist.artist_id'
song_join_artist_df = pd.read_sql(song_select,conn)

#change the data for following merge operation
song_join_artist_df['duration'] = song_join_artist_df['duration'].astype('object')

In [45]:
song_join_artist_df

,song_id,artist_id,name,title,duration
0,SOBBUGU12A8C13E95D,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,Setting Fire to Sleeping Giants,207.778
1,SOGNCJP12A58A80271,ARB29H41187B98F0EF,Terry Callier,Do You Finally Need A Friend,342.569
2,SOPSWQW12A6D4F8781,ARGIWFO1187B9B55B7,Five Bolt Main,Made Like This (Live),225.097
3,SOVYKGO12AB0187199,ARH4Z031187B9A71F2,Faye Adams,Crazy Mixed Up World,156.395
4,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,Elena,Setanta matins,269.583
5,SOGXHEG12AB018653E,AR0RCMP1187FB3F427,Billie Jo Spears,It Makes No Difference Now,133.329
6,SOYTPEP12AB0180E7B,ARAJPHH1187FB5566A,The Shangri-Las,Twist and Shout,164.806
7,SOBLGCN12AB0183212,AR47JEX1187B995D81,SUE THOMPSON,James (Hold The Ladder Steady),124.865
8,SOBZBAZ12A6D4F8742,AROUOZZ1187B9ABE51,Willie Bobo,Spanish Grease,168.254
9,SOOLYAZ12A6701F4A6,AREBBGV1187FB523D2,Mike Jones (Featuring CJ_ Mello & Lil' Bran),Laws Patrolling (Album Version),173.662


In [46]:
#reset the song_join_artist_df index for following merge operation
song_join_artist_df.index = list(song_join_artist_df.index)
song_join_artist_join_logfile_df = pd.merge(log_file_df, song_join_artist_df, left_on=['artist', 'song', 'length'], right_on=['name', 'title', 'duration'], how='left')
song_join_artist_join_logfile_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,song,status,ts,userAgent,userId,song_id,artist_id,name,title,duration
0,None,Logged In,Celeste,F,0,Williams,None,free,"Klamath Falls, OR",GET,...,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,NaN,NaN,NaN,NaN,NaN
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.1604,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,...,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,NaN,NaN,NaN,NaN,NaN
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.159,free,"Klamath Falls, OR",PUT,...,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,NaN,NaN,NaN,NaN,NaN
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.226,free,"Klamath Falls, OR",PUT,...,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,NaN,NaN,NaN,NaN,NaN
4,None,Logged In,Jacqueline,F,0,Lynch,None,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,...,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,NaN,NaN,NaN,NaN,NaN


In [37]:
#song_join_artist_join_logfile_df = pd.merge(log_file_df, song_join_artist_df, left_on=['artist', 'song', 'length'], right_on=['name', 'title', 'duration'], how='left'
#log_file_df.dtypes
song_join_artist_df['duration'] = song_join_artist_df['duration'].astype('object')

In [38]:
#retrieve the column this project needs
songplay_df = song_join_artist_join_logfile_df.loc[:,['ts', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent']]

#clean the column name
songplay_df.index = range(1,len(songplay_df)+1)
songplay_df = songplay_df.reset_index()
songplay_df.columns = ['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent']
songplay_df = songplay_df.fillna(value='None')
songplay_df = songplay_df.replace('', 'None')

In [39]:
songplay_df.head()

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,1,1541990217796,53,free,None,None,438,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,2,1541990258796,10,free,None,None,345,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,3,1541990264796,53,free,None,None,438,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
3,4,1541990541796,53,free,None,None,438,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,5,1541990714796,29,paid,None,None,389,"Atlanta-Sandy Springs-Roswell, GA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    
    
songplay_list = songplay_df.values.tolist()

for row in songplay_list:
    cur.execute(time_table_insert, row)
    conn.commit()

In [40]:
songplay_list = songplay_df.values.tolist()

for row in songplay_list:
    cur.execute(songplay_table_insert, row)
    conn.commit()

In [ ]:
cur.execute("select * from songplay")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [1]:
conn.close()

NameError: name 'conn' is not defined

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.